## Check what GPU you got
Click the Runtime dropdown at the top of the page, then Change Runtime Type and confirm the instance type is GPU.

Check the output of !nvidia-smi to make sure you've been allocated a Tesla P100.

In [0]:
!nvidia-smi

## Pre-requisites
Mount the source code and set up Spacy

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.chdir("/content/drive/My Drive/Translator/src")
!ls

!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

!pip3 install 'torchtext==0.5.0'

!pip3 install torch torchvision


## Build the Vocabs
Make the vocabulary from a dataset

In [0]:
%%shell
DATASET=Multi30k
TRAIN=../data/${DATASET}/Training/
TEST=../data/${DATASET}/Testing/

echo "Making the vocabulary"

python3 build-vocabs.py vocab $TRAIN fr ../models/${DATASET}/vocab.french.gz
python3 build-vocabs.py vocab $TRAIN en ../models/${DATASET}/vocab.english.gz

echo "Finished making the vocabulary"

## From English to French
Train the model with attention

In [0]:
%%shell
DATASET=Multi30k
TRAIN=../data/${DATASET}/Training/
TEST=../data/${DATASET}/Testing/

echo "Training the model"

python3 train.py "${TRAIN}" \
    en "../models/${DATASET}/vocab.english.gz" \
    fr "../models/${DATASET}/vocab.french.gz" \
    "../models/${DATASET}/model.en.fr.pt" \
    --source-word-embedding-size 256 \
    --target-word-embedding-size 256 \
    --encoder-num-layers 3 \
    --encoder-num-attention-heads 8 \
    --encoder-pf-size 512 \
    --encoder-dropout 0.1 \
    --decoder-num-layers 3 \
    --decoder-num-attention-heads 8 \
    --decoder-pf-size 512 \
    --decoder-dropout 0.1 \
    --patience 5 \
    --train-val-ratio 0.75 \
    --batch-size 64 \
    --seed 1 \
    --device cuda \
    --resume-from-checkpoint "../models/${DATASET}/checkpoint.en.fr.pt" \
    --save-checkpoint-to "../models/${DATASET}/checkpoint.en.fr.pt" \

echo "Finished training"

Test the model with attention from English to French

In [0]:
%%shell
DATASET=Multi30k
TRAIN=../data/${DATASET}/Training/
TEST=../data/${DATASET}/Testing/

echo "Testing the model with attention from English to French"

python3 test.py $TEST \
    en ../models/${DATASET}/vocab.english.gz \
    fr ../models/${DATASET}/vocab.french.gz \
    ../models/${DATASET}/model.en.fr.pt \
    --source-word-embedding-size 256 \
    --target-word-embedding-size 256 \
    --encoder-num-layers 3 \
    --encoder-num-attention-heads 8 \
    --encoder-pf-size 512 \
    --encoder-dropout 0.1 \
    --decoder-num-layers 3 \
    --decoder-num-attention-heads 8 \
    --decoder-pf-size 512 \
    --decoder-dropout 0.1 \
    --device cuda \
    --batch-size 64 \

echo "Finished testing"

## From French to English
Train the model with attention

In [0]:
%%shell
DATASET=Multi30k
TRAIN=../data/${DATASET}/Training/
TEST=../data/${DATASET}/Testing/

echo "Training the model from French to English"

python3 runner/train.py "${TRAIN}" \
    fr "../models/${DATASET}/vocab.french.gz" \
    en "../models/${DATASET}/vocab.english.gz" \
    "../models/${DATASET}/model.fr.en.pt" \
    --source-word-embedding-size 256 \
    --target-word-embedding-size 256 \
    --encoder-num-layers 3 \
    --encoder-num-attention-heads 8 \
    --encoder-pf-size 512 \
    --encoder-dropout 0.1 \
    --decoder-num-layers 3 \
    --decoder-num-attention-heads 8 \
    --decoder-pf-size 512 \
    --decoder-dropout 0.1 \
    --patience 5 \
    --train-val-ratio 0.75 \
    --batch-size 64 \
    --seed 1 \
    --device cuda \
    --resume-from-checkpoint "../models/${DATASET}/checkpoint.fr.en.pt" \
    --save-checkpoint-to "../models/${DATASET}/checkpoint.fr.en.pt" \

echo "Finished training"

Test the model with attention from French to English

In [0]:
%%shell
DATASET=Multi30k
TRAIN=../data/${DATASET}/Training/
TEST=../data/${DATASET}/Testing/

echo "Testing the model with attention from French to English"

python3 test.py $TEST \
    fr ../models/${DATASET}/vocab.french.gz \
    en ../models/${DATASET}/vocab.english.gz \
    ../models/${DATASET}/model.fr.en.pt \
    --source-word-embedding-size 256 \
    --target-word-embedding-size 256 \
    --encoder-num-layers 3 \
    --encoder-num-attention-heads 8 \
    --encoder-pf-size 512 \
    --encoder-dropout 0.1 \
    --decoder-num-layers 3 \
    --decoder-num-attention-heads 8 \
    --decoder-pf-size 512 \
    --decoder-dropout 0.1 \
    --device cuda \
    --batch-size 64 \

echo "Finished testing"